In [ ]:
import pandas as pd
import sqlite3

import json

import json

def json_to_sqlite(json_file, table_name, database_name):
    # Charger le fichier JSON dans un DataFrame
    with open(json_file, 'r', encoding='UTF-8') as file:
        data_json = json.load(file)
    df = pd.json_normalize(data_json)


    for col in df.columns:
        if df[col].apply(type).eq(list).any() or df[col].apply(type).eq(dict).any():
            df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x)

    # Se connecter à la base de données SQLite
    conn = sqlite3.connect(database_name)

    #Créer une table dans la base de données utilisant la fonction to_sql de pandas
    df.to_sql(table_name, conn, index=False, if_exists='replace')

    # Fermer la connexion à la base de données
    conn.close()

def excel_to_sqlite(excel_file, table_name, database_name):

    df = pd.read_excel(excel_file)
    data = pd.DataFrame(df)

    # Se connecter à la base de données SQLite
    conn = sqlite3.connect(database_name)

    #Créer une table dans la base de données utilisant la fonction to_sql de pandas
    data.to_sql(table_name, conn, index=False, if_exists='replace')

    # Fermer la connexion à la base de données
    conn.close()

def csv_to_sqlite(csv_file, table_name, database_name):
    # Charger le fichier CSV dans un DataFrame
    df = pd.read_csv(csv_file, delimiter=';', encoding='UTF-8')
    data=pd.DataFrame(df)

    # Se connecter à la base de données SQLite
    conn = sqlite3.connect(database_name)

    #Créer une table dans la base de données utilisant la fonction to_sql de pandas
    data.to_sql(table_name, conn, index=False, if_exists='replace')

    conn.close()

#Créer une table dans la base de données pour les differentes source des données disponible
csv_file_path1 = 'prix-des-carburants-j-7.csv'
table_name = 'Prix'
database_name = 'test.db'
csv_to_sqlite(csv_file_path1, table_name, database_name)

excel_file_path2 = 'osm-restaurant-fr@babel.xlsx'
table_name = 'Restaurant'
database_name = 'test.db'
excel_to_sqlite(excel_file_path2, table_name, database_name)

json_file_path3 = 'carte_des_campings_classes.json'
table_name = 'campings'
database_name = 'test.db'
json_to_sqlite(json_file_path3, table_name, database_name)

csv_file_path4 = 'parcs-animaliers.csv'
table_name = 'Parcs'
database_name = 'test.db'
csv_to_sqlite(csv_file_path4, table_name, database_name)
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
sql = """
CREATE VIEW Stationss AS
SELECT cp, price_gazole, price_sp95, price_sp98, name, price_gplc, price_e10, price_e85, address
FROM prix;
"""
sqlParc = """
CREATE VIEW Parcs_view AS
SELECT cp, Type, nom, adresse, web
FROM Parcs;
"""
sqlCamping = """
CREATE VIEW Camping_view AS
SELECT code_postal, adresse, nom_commercial, site_internet
FROM campings;
"""
sqlR = """
CREATE VIEW Restaurant_view AS
SELECT code_commune, name, type, cuisine,phone, opening_hours
FROM Restaurant;
"""
req1= "DROP VIEW IF EXISTS Stationss"
cursor.execute(req1)
cursor.execute(sql)

req2= "DROP VIEW IF EXISTS Parcs_view"
cursor.execute(req2)
cursor.execute(sqlParc)

req3= "DROP VIEW IF EXISTS Camping_view"
cursor.execute(req3)
cursor.execute(sqlCamping)

req4= "DROP VIEW IF EXISTS Restaurant_view"
cursor.execute(req4)
cursor.execute(sqlR)


conn.close()

In [ ]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

print("Bonjour !")
print("Quel est votre choix : ")
print("1- Afficher toutes les stations de votre ville ")
print("2- Afficher le prix du carburant dans toutes les stations de votre ville  ")
print("3- Afficher les informations de tous les parcs proches de votre ville   ")
print("4- Afficher les informations de tous les Campings de votre ville ")
print("5- Afficher les informations de tous les Restaurants de votre ville ")

var = "false"

while var == "false":
    chiffre_saisi = int(input("Veuillez saisir un nombre entre 1 et 5: "))

    if chiffre_saisi >= 1 and chiffre_saisi <= 5:
        var = "true"
    else:
        print("Veuillez saisir un choix entre 1 et 5")

if (chiffre_saisi == 2):
   options_valides = ["price_gazole", "price_sp95", "price_sp98", "name", "price_gplc", "price_e10", "price_e85"]
   Carburant = ""
   while True:  # Demande le code postal
        code_postal = int(input("Veuillez saisir le code postal de votre ville: "))
        while Carburant not in options_valides:
          Carburant= input("Veuillez saisir un des carburants suivants (price_gazole, price_sp95, price_sp98, price_gplc, price_e10, price_e85) : ")
        if Carburant not in options_valides:
          print("Option invalide. Veuillez ecrire la bonne syntaxe.")

        sql1 = f"""
        SELECT name, {Carburant}, address
        FROM Stationss
        WHERE cp = ?;
        """
        cursor.execute(sql1, (code_postal,))
       # Récupérez tous les résultats
        resultats = cursor.fetchall()
        if not resultats:  # Si aucun résultat n'est trouvé
          print("Aucune station trouvée pour ce code postal. Veuillez réessayer.")
        else:  # Si des résultats sont trouvés
          break
   for resultat in resultats:
      print(f"Station: {resultat[0]}, Prix du Gazole: {resultat[1]},  address: {resultat[2]}  ")
   conn.commit()
elif (chiffre_saisi==1) :
   while True:
        code_postal = int(input("Veuillez saisir le code postal de votre ville: "))
        sql1 = f"""
        SELECT name , address
        FROM Stationss
        WHERE cp = ?;
        """
        cursor.execute(sql1, (code_postal,))

        resultats = cursor.fetchall()
        if not resultats:  # Si aucun résultat n'est trouvé
          print("Aucune station trouvée pour ce code postal. Veuillez réessayer.")
        else:
          break
   for resultat in resultats:
      print(f"Station: {resultat[0]},  address: {resultat[1]}  ")
   conn.commit()

elif (chiffre_saisi==3) :
    while True:
        code_postal = int(input("Veuillez saisir le code postal de votre ville: "))
        sql1 = f"""
        SELECT Type, nom, adresse, web
        FROM Parcs_view
        WHERE cp = ?;
        """
        cursor.execute(sql1, (code_postal,))

        resultats = cursor.fetchall()
        if not resultats:
          print("Aucune Parc trouvé pour ce code postal. Veuillez réessayer.")
        else:
          break
    for resultat in resultats:
      print(f"Type: {resultat[0]},  nom : {resultat[1]}, ,  adress : {resultat[2]}, site : {resultat[3]}   ")
    conn.commit()
elif (chiffre_saisi==4) :
    while True:
        code_postal = int(input("Veuillez saisir le code postal de votre ville: "))
        sql1 = f"""
        SELECT nom_commercial, adresse, site_internet
        FROM Camping_view
        WHERE code_postal = ?;
        """
        cursor.execute(sql1, (code_postal,))
        resultats = cursor.fetchall()
        if not resultats:
          print("Aucun Camping trouvé pour ce code postal. Veuillez réessayer.")
        else:
          break
    for resultat in resultats:
      print(f" {resultat[0]} , adresse: {resultat[1]} , site: {resultat[2]}   ")
    conn.commit()
elif (chiffre_saisi==5) :
    while True:
        code_postal = int(input("Veuillez saisir le code postal de votre ville: "))
        sql1 = f"""
        SELECT name, type, cuisine,phone, opening_hours
        FROM Restaurant_view
        WHERE code_commune = ?;
        """
        cursor.execute(sql1, (code_postal,))

        resultats = cursor.fetchall()
        if not resultats:
          print("Aucun Restaurant trouvé pour ce code postal. Veuillez réessayer.")
        else:
          break
    for resultat in resultats:
        resultat_liste = list(resultat)

        for i, valeur in enumerate(resultat_liste):
            if valeur is None or str(valeur).lower() == "none":
                resultat_liste[i] = "non renseigné"

        print(f" {resultat_liste[0]} , Type: {resultat_liste[1]} , cuisine : {resultat_liste[2]}, Telephone {resultat_liste[3]}, Horaire d'ouverture: {resultat_liste[4]} ")
        conn.commit()

